In [38]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()



In [39]:
print ("x_train row 0",x_train.shape,x_train[0])
print ("y_train row 0",y_train[0])
num_pixels = x_train.shape[1] * x_train.shape[2]*x_train.shape[3]
x_train = x_train.reshape(x_train.shape[0], num_pixels).astype('float32')
x_test = x_test.reshape(x_test.shape[0], num_pixels).astype('float32')
x_train=x_train/255
x_test=x_test/255
print(num_pixels)
print (x_train[0])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]
print(num_classes)


x_train row 0 (50000, 32, 32, 3) [[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]
y_train row 0 [6]
3072
[0.23137255 0.24313726 0.24705882 ... 0.48235294 0.36078432 0.28235295]
10


In [51]:
def classification_model():
  model = Sequential()
  model.add(tf.keras.Input(shape=(num_pixels,)))
  model.add(Dense(300, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

The prior cell creates a sequential model with 937,460 trainable parameters. ((3072+1)*300) + (300*50) +(50*10) +10 

In [52]:
model1 = classification_model()
model1.summary()
model1.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=64, epochs=20, verbose=2)


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 300)               921900    
                                                                 
 dense_32 (Dense)            (None, 50)                15050     
                                                                 
 dense_33 (Dense)            (None, 10)                510       
                                                                 
Total params: 937,460
Trainable params: 937,460
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
782/782 - 4s - loss: 1.9329 - accuracy: 0.3039 - val_loss: 1.8446 - val_accuracy: 0.3414 - 4s/epoch - 5ms/step
Epoch 2/20
782/782 - 3s - loss: 1.7567 - accuracy: 0.3774 - val_loss: 1.7967 - val_accuracy: 0.3586 - 3s/epoch - 4ms/step
Epoch 3/20
782/782 - 4s - loss: 1.6793 - accuracy: 0.4050 - val_los

In [53]:
scores = model1.evaluate(x_test, y_test, verbose=0)
print('Model Accuracy: {} \n Error: {}'.format(scores[1], 1 - scores[1]))

Model Accuracy: 0.5091000199317932 
 Error: 0.4908999800682068


The training set reached an accuracy of 54.55%.  The validation accuracy was a little lower at 50.9%.  I tried using more or less neurons in the hidden layers and that did not see to have a great effect on the numbers. Over multiple trials with different number of neurons, The best validation accuracy I could get was a little under 54% (40 epochs) and the worst was around 46.5%. 

In [54]:
model2 = classification_model()
model2.summary()
model2.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=64, epochs=40, verbose=2)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 300)               921900    
                                                                 
 dense_35 (Dense)            (None, 50)                15050     
                                                                 
 dense_36 (Dense)            (None, 10)                510       
                                                                 
Total params: 937,460
Trainable params: 937,460
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
782/782 - 4s - loss: 1.9259 - accuracy: 0.3095 - val_loss: 1.8541 - val_accuracy: 0.3227 - 4s/epoch - 5ms/step
Epoch 2/40
782/782 - 4s - loss: 1.7561 - accuracy: 0.3787 - val_loss: 1.7412 - val_accuracy: 0.3748 - 4s/epoch - 5ms/step
Epoch 3/40
782/782 - 4s - loss: 1.6771 - accuracy: 0.4065 - val_los

In [55]:
scores = model2.evaluate(x_test, y_test, verbose=0)
print('Model Accuracy: {} \n Error: {}'.format(scores[1], 1 - scores[1]))

Model Accuracy: 0.527400016784668 
 Error: 0.47259998321533203


For the 40 Epoch model, the accuracy on the training set did improve by 7.3%  but the validation set accuracy increased by only 1.8% which does not seem like a really great return on double the compute time. 

In [56]:
labels_map = {
  0: 'Airplane',
  1: 'Automobile',
  2: 'Bird',
  3: 'Cat',
  4: 'Deer',
  5: 'Dog',
  6: 'Frog',
  7: 'Horse',
  8: 'Ship',
  9: 'Truck',
}
Z=np.random.randint(10000)
predicted_vector1 = model1(np.expand_dims(x_test[Z], axis=0))
predicted_vector2 = model2(np.expand_dims(x_test[Z], axis=0))
predicted_index = np.argmax(predicted_vector1)
predicted_index2 = np.argmax(predicted_vector2)
print ("The 20 Epoch Neural Net Thinks its a:",labels_map[predicted_index] )
print ("The 40 Epoch Neural Net Thinks its a:",labels_map[predicted_index2])
print ("The image is a:",labels_map[np.argmax(y_test[Z])])

The 20 Epoch Neural Net Thinks its a: Dog
The 40 Epoch Neural Net Thinks its a: Dog
The image is a: Dog


I spent a lot of time this weekend trying to tweak the models (number of neurons in each layer) to see if I could find some optimal number but I couldn't acheive better than 54% accuracy on the validation set. I was able to get the trainng accuracy up ove 70% a few times but the validation still was around 53.5%.  